# multi-head Latent Attention

![MLA](./img/MLA.png)

## Preliminaries: Standard Multi-Head Attention
- d is embedding dimension 
- $n_h$ number of attention heads 
- $d_h$ dimension per head
- $h_t \in \mathbb{R}^{d} $ be the attention input of the $t$-th token at an attention layer

设输入token，标准的 MHA 首先通过三个投影矩阵（proj matrices）：$W^{Q}, W^{K}, W^{V} \in \mathbb{R}^{d_h n_h \times d}$ 生成 $\mathbf{q_t}, \mathbf{k_t}, \mathbf{v_t} \in \mathbb{R}^{d_h n_h}$：

$$\mathbf{q_t} = W^{Q}\mathbf{h_t},$$
$$\mathbf{k_t} = W^{K}\mathbf{h_t},$$
$$\mathbf{v_t} = W^{V}\mathbf{h_t},$$

Then, $\mathbf{q}_t, \mathbf{k}_t, \mathbf{v}_t$ will be sliced into $n_h$ heads for the multi-head attention computation:

$$
\begin{align*}
[\mathbf{q}_{t,1}; \mathbf{q}_{t,2}; \ldots; \mathbf{q}_{t,n_h}] &= \mathbf{q}_t, \\
[\mathbf{k}_{t,1}; \mathbf{k}_{t,2}; \ldots; \mathbf{k}_{t,n_h}] &= \mathbf{k}_t, \\
[\mathbf{v}_{t,1}; \mathbf{v}_{t,2}; \ldots; \mathbf{v}_{t,n_h}] &= \mathbf{v}_t,
\end{align*}
$$

$$
\mathbf{o}_{t,i} = \sum_{j=1}^{t} \text{Softmax}_j \left( \frac{\mathbf{q}_{t,i}^T \mathbf{k}_{j,i}}{\sqrt{d_h}} \right) \mathbf{v}_{j,i},
$$

$$
\mathbf{u}_t = \mathbf{W}^O [\mathbf{o}_{t,1}; \mathbf{o}_{t,2}; \ldots; \mathbf{o}_{t,n_h}],
$$
- $\mathbf{q_{t,i}}, \mathbf{k_{t,i}}, \mathbf{v_{t,i}} \in \mathbb{R}^{d_h}$ 是第i个attention head 。
- $W^O \in \mathbb{R}^{d \times d_h n_h}$ 表示输出的投影矩阵
- **Total KV-cache**: $2 * n_h * d_h * l$ 对于整个序列的token都需要缓存k和v(l is number of layer)

## Low-Rank Key-Value Joint Compression

$$
\begin{align*}
\mathbf{c}_t^{KV} &= \mathbf{W}^{DKV} \mathbf{h}_t, \\
\mathbf{k}_t^C &= \mathbf{W}^{UK} \mathbf{c}_t^{KV}, \\
\mathbf{v}_t^C &= \mathbf{W}^{UV} \mathbf{c}_t^{KV},
\end{align*}
$$

- $c_t^{KV} \in \mathbb{R}^{d_c}$ 是一个 compressed latent 向量 (for k, v)，并且 $d_c (\ll d_h n_h)$
- $\mathbf{W}^{DKV} \in \mathbb{R}^{d_c \times d}$ 是一个降维的投影矩阵
- $\mathbf{W}^{UK}, \mathbf{W}^{UV} \in \mathbb{R}^{d_h n_h \times d_c}$ 是升维投影矩阵
- **KV cache** 这样就下降到了：$d_c * l$


RoPE解耦策略：引入解耦的qk携带位置编码

 - $\mathbf{q}_{t,i}^R \in \mathbb{R}^{d_h^R}$ and a shared key $\mathbf{k}_t^R \in \mathbb{R}^{d_h^R}$ to carry RoPE, 
 - $d_h^R$ denotes the per-head dimension of the decoupled queries and key.

$$
[\mathbf{q}_{t,1}^R; \mathbf{q}_{t,2}^R; \ldots; \mathbf{q}_{t,n_h}^R] = \mathbf{q}_t^R = \text{RoPE}(W^{QR} \mathbf{c}_t^Q), 
$$

$$
\mathbf{k}_t^R = \text{RoPE}(W^{KR} \mathbf{h}_t), 
$$

$$
\mathbf{q}_{t,i} = [\mathbf{q}_{t,i}^C; \mathbf{q}_{t,i}^R],
$$

$$
\mathbf{k}_{t,i} = [\mathbf{k}_{t,i}^C; \mathbf{k}_t^R], 
$$

$$
\mathbf{o}_{t,i} = \sum_{j=1}^{t} \text{Softmax}_j\left(\frac{\mathbf{q}_{t,i}^T \mathbf{k}_{j,i}}{\sqrt{d_h + d_h^R}}\right) \mathbf{v}_{j,i}^C 
$$

$$
\mathbf{u}_t = W^O [\mathbf{o}_{t,1}; \mathbf{o}_{t,2}; \ldots; \mathbf{o}_{t,n_h}], 
$$
- 注解：
    - $W^{QR} \in \mathbb{R}^{d_h^R n_h \times d_c'}$ and $W^{KR} \in \mathbb{R}^{d_h^R \times d}$ 为解耦q, k矩阵
    - $\text{RoPE}(\cdot)$ 为旋转位置编码算子
    - $[\cdot; \cdot]$ 为concat算子

# MLA 整体流程

MLA 的流程涉及多个步骤，包括压缩、解耦 RoPE 和注意力计算。假设输入隐藏状态维度为 $d$，注意力头数为 $n_h$，每个头的维度为 $d_h$，KV 压缩维度为 $d_c$（满足 $d_c \ll d_h n_h$），解耦 RoPE 的维度为 $d_h^R$。

---

## 1. 低秩 KV 压缩
- 输入 $h_t \in \mathbb{R}^d$，首先生成压缩的潜在向量：
  $$
  \mathbf{c}_t^{KV} = W^{DKV}h_t
  $$
  其中 $W^{DKV} \in \mathbb{R}^{d_c \times d}$，因此 $\mathbf{c}_t^{KV} \in \mathbb{R}^{d_c}$。

- 然后通过上投影矩阵生成压缩的 Key 和 Value：
  $$
  \mathbf{k}_t^C = W^{UK} \mathbf{c}_t^{KV}, \quad \mathbf{v}_t^C = W^{UV} \mathbf{c}_t^{KV}
  $$
  其中 $W^{UK}, W^{UV} \in \mathbb{R}^{d_h n_h \times d_c}$，因此 $\mathbf{k}_t^C, \mathbf{v}_t^C \in \mathbb{R}^{d_h n_h}$。  
  这些向量通常被重塑为 $n_h \times d_h$ 用于多头注意力。

---

## 2. 查询压缩（可选，用于对称性）
- 查询压缩：
  $$
  \mathbf{c}_t^Q = W^{DQ}h_t
  $$
  其中 $W^{DQ} \in \mathbb{R}^{d_c' \times d}$，$\mathbf{c}_t^Q \in \mathbb{R}^{d_c'}$（通常 $d_c' = d_c$）。

- 上投影：
  $$
  \mathbf{q}_t^C = W^{UQ} \mathbf{c}_t^Q
  $$
  其中 $W^{UQ} \in \mathbb{R}^{d_h n_h \times d_c'}$，因此 $\mathbf{q}_t^C \in \mathbb{R}^{d_h n_h}$。

---

## 3. 解耦 RoPE
为了解决 RoPE 与低秩压缩的兼容性问题，引入解耦的查询和键。

- **解耦查询**  
  通过矩阵 $W^{QR} \in \mathbb{R}^{d_h^R n_h \times d_c'}$ 生成：
  $$
  \mathbf{q}_t^R = \text{RoPE}(W^{QR} \mathbf{c}_t^Q)
  $$
  其中 $\mathbf{q}_t^R \in \mathbb{R}^{d_h^R n_h}$，被重塑为 $n_h \times d_h^R$。RoPE 应用于每个头的位置。

- **解耦键**  
  通过矩阵 $W^{KR} \in \mathbb{R}^{d_h^R \times d}$ 生成共享键：
  $$
  \mathbf{k}_t^R = \text{RoPE}(W^{KR}h_t)
  $$
  其中 $\mathbf{k}_t^R \in \mathbb{R}^{d_h^R}$，所有头共享同一个解耦键。RoPE 同样应用。

- **拼接操作**
  - 对于每个头 $i$，查询拼接：  
    $\mathbf{q}_{t,i} = [\mathbf{q}_{t,i}^C; \mathbf{q}_{t,i}^R]$  
    → 每个头的查询维度变为 $d_h + d_h^R$。
  - 对于每个头 $i$，键拼接：  
    $\mathbf{k}_{t,i} = [\mathbf{k}_{t,i}^C; \mathbf{k}_{t,i}^R]$  
    → 每个头的键维度变为 $d_h + d_h^R$（$\mathbf{k}_t^R$ 被广播到所有头）。

---

## 4. 注意力计算
- 注意力得分（每个头 $i$）：
  $$
  \text{score} = \frac{\mathbf{q}_{t,i}^T \mathbf{k}_{j,i}}{\sqrt{d_h + d_h^R}}
  $$

- 输出（每个头）：
  $$
  \mathbf{o}_{t,i} = \sum_{j=1}^{t} \text{Softmax}_j(\text{score}) \mathbf{v}_{j,i}^C
  $$
  其中 $\mathbf{v}_{j,i}^C$ 是压缩 Value 的第 $i$ 个头，维度 $d_h$。

- 拼接所有头的输出并线性投影：
  $$
  \mathbf{u}_t = W^O [\mathbf{o}_{t,1}; \mathbf{o}_{t,2}; \ldots; \mathbf{o}_{t,n_h}]
  $$
  其中 $W^O \in \mathbb{R}^{d \times d_h n_h}$。

---

## 5. KV 缓存
推理时，需要缓存每层的两个向量：
- 压缩潜在向量 $\mathbf{c}_t^{KV} \in \mathbb{R}^{d_c}$
- 解耦键 $\mathbf{k}_t^R \in \mathbb{R}^{d_h^R}$

因此，每个 token 每层的缓存维度为：
$$
d_c + d_h^R
$$

对于序列长度 $S$ 和层数 $L$，总缓存大小为：
$$
S \times (d_c + d_h^R) \times L
$$


# 关于矩阵吸收

- Key 吸收：
  - 在计算注意力得分时，需要计算压缩查询和压缩键的点积：$(\mathbf{q}_t^C)^T \mathbf{k}_j^C$。
  - 由于 $\mathbf{q}_t^C = W^{UQ} \mathbf{c}_t^Q$ 和 $\mathbf{k}_j^C = W^{UK} \mathbf{c}_j^{KV}$，我们有：
    $$
    (\mathbf{q}_t^C)^T \mathbf{k}_j^C = (\mathbf{c}_t^Q)^T (W^{UQ})^T W^{UK} \mathbf{c}_j^{KV}
    $$
  - 预先计算矩阵 $W^{K'} = (W^{UQ})^T W^{UK} \in \mathbb{R}^{d_c' \times d_c}$，则点积可通过低维向量计算：
    $$
    (\mathbf{q}_t^C)^T \mathbf{k}_j^C = (\mathbf{c}_t^Q)^T W^{K'} \mathbf{c}_j^{KV}
    $$
  - 这样无需显式生成高维的 $\mathbf{q}_t^C$ 和 $\mathbf{k}_j^C$。

- Value 吸收：
  - 注意力输出后需要投影。注意力输出是加权和的 $\mathbf{v}_j^C$，而 $\mathbf{v}_j^C = W^{UV} \mathbf{c}_j^{KV}$。
  - 输出投影时：
    $$
    \mathbf{u}_t = W^O \left( \sum_j \alpha_j \mathbf{v}_j^C \right) = W^O \left( \sum_j \alpha_j W^{UV} \mathbf{c}_j^{KV} \right) = (W^O W^{UV}) \left( \sum_j \alpha_j \mathbf{c}_j^{KV} \right)
    $$
  - 预先计算矩阵 $W^{O'} = W^O W^{UV} \in \mathbb{R}^{d \times d_c}$，则输出可直接从低维加权和计算：
    $$
    \mathbf{u}_t = W^{O'} \left( \sum_j \alpha_j \mathbf{c}_j^{KV} \right)
    $$